In [1]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [82]:
base_url = 'https://en.wikipedia.org'
url = 'https://en.wikipedia.org/wiki/List_of_international_airports_by_country'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'lxml')
provinces_states = []

## Pulling Canadian Data

In [85]:
canadian_rows = soup.find(id='Canada').parent.find_next_sibling('table').contents[1].find_all('tr')
for row_ind in range(1, len(canadian_rows)):
    follow_page = canadian_rows[row_ind].find_all('a')[0]['href']
    url_follow = base_url + follow_page
    soup_follow = BeautifulSoup(urllib.request.urlopen(url_follow))
    province_exists = soup_follow.find('a', text='Province', class_=False)
    if province_exists:
        prov_element = province_exists.parent.next_sibling.contents[0]
        if prov_element.name == 'a':
            provinces_states.append(prov_element.getText())

In [84]:
provinces_states

['Alberta',
 'Alberta',
 'Newfoundland and Labrador',
 'Nova Scotia',
 'Ontario',
 'British Columbia',
 'Ontario',
 'Ontario',
 'Quebec',
 'Saskatchewan',
 'Saskatchewan',
 'Ontario',
 'British Columbia',
 'Manitoba']

## United States

This is giving me cancer... nothing is standard

In [112]:
us_rows = soup.find(id='United_States').parent.find_next_sibling('table').contents[1].find_all('tr')
for row_ind in range(1, len(us_rows)):
    follow_page = us_rows[row_ind].find_all('a')[0]['href']
    url_follow = base_url + follow_page
    soup_follow = BeautifulSoup(urllib.request.urlopen(url_follow))
    state_exists = soup_follow.find(text='State', class_=False)
    print(state_exists)
    if state_exists.name == 'a':
        print("fuck1")
        #prov_element = state_exists.parent.next_sibling.a
        #print(prov_element)
        #if prov_element:
            #provinces_states.append(prov_element.getText())
            #print(prov_element.getText())
    elif state_exists.name == 'th':
        print("fuck2")
        if state_exists.find_next('a'):
            print("found hyperlink")
            print(state_exists.find_next('a'))
        else:
            print(state_exists.next_sibling)

<th scope="row"><a href="/wiki/U.S._state" title="U.S. state">State</a></th>
fuck2
found hyperlink
<a href="/wiki/U.S._state" title="U.S. state">State</a>
<th scope="row">State</th>
fuck2
found hyperlink
<a href="/wiki/New_York_(state)" title="New York (state)">New York</a>
<th scope="row">State</th>
fuck2
found hyperlink
<a href="/wiki/List_of_counties_in_New_Mexico" title="List of counties in New Mexico">County</a>
<th scope="row">State</th>
fuck2
found hyperlink
<a href="/wiki/Alaska" title="Alaska"><img alt="Alaska" class="thumbborder" data-file-height="1000" data-file-width="1416" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Flag_of_Alaska.svg/21px-Flag_of_Alaska.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Flag_of_Alaska.svg/33px-Flag_of_Alaska.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Flag_of_Alaska.svg/43px-Flag_of_Alaska.svg.png 2x" width="21"/></a>
<th scope="row"><a href="/wiki/U.S._stat

KeyboardInterrupt: 

In [127]:
df = pd.DataFrame({'state/province': provinces_states})
df.groupby('state/province')['state/province'].agg('count')

state/province
Alberta                      4
British Columbia             4
Manitoba                     2
Newfoundland and Labrador    2
Nova Scotia                  2
Ontario                      8
Quebec                       2
Saskatchewan                 4
Name: state/province, dtype: int64